In [8]:
import re
from typing import Dict, Any, Callable, Union
import numpy as np
import scipy.stats as stats
import sympy as sp
from dataclasses import dataclass

@dataclass
class HypothesisResult:
    equation: str
    result: float
    p_value: float
    is_significant: bool
    operator: str
    threshold: float

def parse_hypothesis(hypothesis: str) -> tuple:
    """Parse a hypothesis string into left side, operator, and right side."""
    operators = {'>': np.greater, '<': np.less, '>=': np.greater_equal, '<=': np.less_equal, '==': np.equal}
    for op in operators:
        if op in hypothesis:
            left, right = hypothesis.split(op)
            return left.strip(), op, right.strip()
    raise ValueError("No valid operator found in hypothesis")

def evaluate_expression(expr: str, variables: Dict[str, Any]) -> float:
    """Evaluate a mathematical expression using SymPy."""
    return float(sp.sympify(expr).evalf(subs=variables))

def statistical_test(result: float, operator: str, threshold: float) -> float:
    """Perform a statistical test and return the p-value."""
    if operator in ['>', '>=']:
        return 1 - stats.norm.cdf(result)
    elif operator in ['<', '<=']:
        return stats.norm.cdf(result)
    else:  # For '=='
        return 2 * (1 - stats.norm.cdf(abs(result)))

def advanced_hypothesis(hypothesis: str, variables: Dict[str, Any] = None, alpha: float = 0.05) -> HypothesisResult:
    """
    Evaluate a statistical hypothesis with advanced features.

    Args:
    - hypothesis: A string representing the hypothesis (e.g., "x > 0")
    - variables: A dictionary of variables to use. If None, uses globals().
    - alpha: Significance level for hypothesis testing

    Returns:
    - HypothesisResult object containing the evaluation details
    """
    if variables is None:
        variables = globals()

    left, operator, right = parse_hypothesis(hypothesis)
    
    left_value = evaluate_expression(left, variables)
    right_value = evaluate_expression(right, variables)
    
    result = left_value - right_value
    p_value = statistical_test(result, operator, right_value)
    is_significant = p_value < alpha

    hypothesis_result = HypothesisResult(
        equation=hypothesis,
        result=result,
        p_value=p_value,
        is_significant=is_significant,
        operator=operator,
        threshold=right_value
    )

    print(f"Hypothesis: {hypothesis}")
    print(f"Result: {result:.4f}")
    print(f"P-value: {p_value:.4f}")
    print(f"Significant at α={alpha}: {'Yes' if is_significant else 'No'}")
    print(f"{'✅' if is_significant else '❌'}")
    print()

    return hypothesis_result

# Example usage
y = 2.0
z = 0.3

advanced_hypothesis('x > y')
advanced_hypothesis('x + y < 5')
advanced_hypothesis('z == 0')

SympifyError: Sympify of expression 'could not parse 'Automatically created module for IPython interactive environment'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

In [20]:
import re
from typing import Dict, Any, Union
import numpy as np
import scipy.stats as stats
from dataclasses import dataclass

@dataclass
class HypothesisResult:
    equation: str
    result: float
    operator: str
    threshold: float

def parse_hypothesis(hypothesis: str) -> tuple:
    """Parse a hypothesis string into left side, operator, and right side."""
    operators = {'>': np.greater, '<': np.less, '>=': np.greater_equal, '<=': np.less_equal, '==': np.equal}
    for op in operators:
        if op in hypothesis:
            left, right = hypothesis.split(op)
            return left.strip(), op, right.strip()
    raise ValueError("No valid operator found in hypothesis")

def safe_eval(expr: str, variables: Dict[str, Any]) -> float:
    """Safely evaluate a mathematical expression."""
    # Replace variable names with their values
    for var, value in variables.items():
        expr = expr.replace(var, str(value))
    # Use eval with a limited set of allowed names
    allowed_names = {
        'abs': abs, 'max': max, 'min': min, 'pow': pow,
        'round': round, 'sum': sum
    }
    return float(eval(expr, {"__builtins__": None}, allowed_names))


def advanced_hypothesis(hypothesis: str, variables: Dict[str, Any] = None, alpha: float = 0.05) -> HypothesisResult:
    """
    Evaluate a statistical hypothesis with advanced features.

    Args:
    - hypothesis: A string representing the hypothesis (e.g., "x > 0")
    - variables: A dictionary of variables to use. If None, uses globals().
    - alpha: Significance level for hypothesis testing

    Returns:
    - HypothesisResult object containing the evaluation details
    """
    if variables is None:
        variables = globals()

    try:
        left, operator, right = parse_hypothesis(hypothesis)
        print(dict(left=left, operator=operator, right=right))
        left_value = safe_eval(left, variables)
        right_value = safe_eval(right, variables)
        
        result = left_value - right_value

        hypothesis_result = HypothesisResult(
            equation=hypothesis,
            result=result,
            operator=operator,
            threshold=right_value
        )


        print(f"Hypothesis: {hypothesis}")
        print(f"            {left_value} {operator} {right_value}")
        print(f"Residual: {result:.4f}")
        print(f"{'✅' if result else '❌'}")
        print()

        return hypothesis_result
    
    except Exception as e:
        print(f"Error evaluating hypothesis: {str(e)}")
        return None

# Example usage
if __name__ == "__main__":
    x = 2.5
    y = 2.0
    z = 0.3

    advanced_hypothesis('x > y')
    advanced_hypothesis('2 * x + y < 5 ** 2')
    advanced_hypothesis('z == 0')
    advanced_hypothesis('(x * y) > z')

{'left': 'x', 'operator': '>', 'right': 'y'}
Hypothesis: x > y
            2.5 > 2.0
Residual: 0.5000
✅

{'left': '2 * x + y', 'operator': '<', 'right': '5 ** 2'}
Hypothesis: 2 * x + y < 5 ** 2
            7.0 < 25.0
Residual: -18.0000
✅

{'left': 'z', 'operator': '==', 'right': '0'}
Hypothesis: z == 0
            0.3 == 0.0
Residual: 0.3000
✅

{'left': '(x * y)', 'operator': '>', 'right': 'z'}
Hypothesis: (x * y) > z
            5.0 > 0.3
Residual: 4.7000
✅



In [158]:
e = sp.sympify("x = 2")
e

e = parse_expr("2 = x")
list(e.free_symbols)
dir(e)
[s.name for s in e.free_symbols]

SympifyError: Sympify of expression 'could not parse 'x = 2'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

In [188]:
import sympy as sp
from typing import Dict, Any

def shypothesis(hypothesis: str, variables: Dict[str, Any] = None, round=3):
    """
    Evaluate a hypothesis using SymPy, showing simplified equation and result.

    Args:
    - hypothesis: A string representing the hypothesis (e.g., "x > y")
    - variables: A dictionary of variables to use. If None, uses locals().
    - round: Number of decimal places to round the result to

    Returns:
    - None (prints the evaluation details)


    @url: https://gist.github.com/wassname/3880184763913cb48a58a669e66a2eda
    """
    if variables is None:
        variables = globals()

    # Parse the hypothesis
    print(f"Hypothesis: {hypothesis}")
    expr = sp.sympify(hypothesis)
    print(f"          : {expr}")
    
    subs = dict()
    for symbol in expr.free_symbols:
        if symbol.name in variables:
            subs[symbol.name] = variables[symbol.name]
    print(f'     Where: {subs}')

    vlhs, vrhs = expr.lhs.evalf(subs=subs), expr.rhs.evalf(subs=subs)
    if round is not None:
        vlhs = vlhs.round(round)
        vrhs = vrhs.round(round)
    print(f"          : {vlhs} {expr.rel_op} {vrhs}")

    # Simplify by moving everything to one side
    simplified = sp.simplify(expr.lhs - expr.rhs)
    print(f"Residual  : = {simplified}")

    # Determine the comparison operator
    op = expr.func

    # Evaluate the simplified expression
    result_value = simplified.evalf(subs=subs)
    if round is not None:
        result_value = result_value.round(round)

    # Determine the result
    result = op(result_value, 0)

    print(f"Residual  : = {result_value} {'✅' if result else '❌'}")
    print()

# Example usage
if __name__ == "__main__":
    x, y, z = sp.symbols('x y z')
    variables = {
        'x': 2.5,
        'y': 2.0,
        'z': 0.3
    }
    x = 2.5
    y = 2
    z = 0.

    shypothesis('x > y', globals())
    shypothesis('x + y < 5', variables)
    shypothesis('sin(x) < cos(y)', variables)
    shypothesis('x**2 + y**2 > z**2', variables)

Hypothesis: x > y
          : x > y
     Where: {'y': 2, 'x': 2.5}
          : 2.500 > 2.00000000000000
Residual  : = x - y
Residual  : = 0.500 ✅

Hypothesis: x + y < 5
          : x + y < 5
     Where: {'y': 2.0, 'x': 2.5}
          : 4.500 < 5.00000000000000
Residual  : = x + y - 5
Residual  : = -0.500 ✅

Hypothesis: sin(x) < cos(y)
          : sin(x) < cos(y)
     Where: {'y': 2.0, 'x': 2.5}
          : 0.598 < -0.416
Residual  : = sin(x) - cos(y)
Residual  : = 1.015 ❌

Hypothesis: x**2 + y**2 > z**2
          : x**2 + y**2 > z**2
     Where: {'y': 2.0, 'z': 0.3, 'x': 2.5}
          : 10.250 > 0.090
Residual  : = x**2 + y**2 - z**2
Residual  : = 10.160 ✅



In [2]:
import re

def remove_fstr(s):
    # then remove the f'
    s = re.sub(r"f'", "'", s)
    return s.replace('}', '').replace('{', '')

def fhypothesis(equation: str, fmt='.3g', variables: dict = None):
    """
    Evaluate a mathematical hypothesis with variables automatically fetched from the global scope.

    Args:
    - equation: A mathematical equation with variables in curly brackets.
    - fmt: Format specifier for the result.
    Returns:
    - None: Prints the hypothesis and its evaluation.
        
    Usage:    
      a = 0.001
      b = -0.02
      fhypothesis('{a}-{b}')
      fhypothesis('{b}-{a}')
      
      > hypothesis:     ( a - b )    > 0 ?
      > working:     ( 0.00 - -0.02 ) = 0.02    > 0    ∴ ✅
      > hypothesis:     ( b - a )    > 0 ?
      > working:     ( -0.02 - 0.00 ) = -0.02    > 0    ∴ ❌

    https://gist.github.com/wassname/3880184763913cb48a58a669e66a2eda
  
    """
    if variables is None:
        variables = globals()
      
    eq_wfmt = equation.replace('}', ':'+fmt+'}')
    eq_pln = remove_fstr(equation)
    res = eval(eq_pln, variables)
    res_fmt = f"{res:{fmt}}"
    sym = '✅' if bool(res>0) else '❌'
    wrk_fmt = "f'( "+eq_wfmt+f" ) = {res_fmt}\t> 0\t∴ {sym}'"
    print(f'hypothesis: ( {eq_pln} )\t> 0 ?')
    print('working:   ', eval(wrk_fmt, variables))
    print()

a = 1e-9
b = -0.02
fhypothesis('{a} - {b}')
fhypothesis('{b} - {a}')

hypothesis: ( a - b )	> 0 ?
working:    ( 1e-09 - -0.02 ) = 0.02	> 0	∴ ✅

hypothesis: ( b - a )	> 0 ?
working:    ( -0.02 - 1e-09 ) = -0.02	> 0	∴ ❌

